In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
data = pd.read_csv('twitter_data.csv', encoding='latin1')

In [14]:
data.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech:confidence,_created_at,orig__golden,orig__last_judgment_at,orig__trusted_judgments,orig__unit_id,orig__unit_state,_updated_at,orig_does_this_tweet_contain_hate_speech,does_this_tweet_contain_hate_speech_gold,does_this_tweet_contain_hate_speech_gold_reason,does_this_tweet_contain_hate_speechconfidence,tweet_id,tweet_text
0,853718217,True,golden,86,NaN,The tweet uses offensive language but not hate...,0.6013,NaN,True,NaN,0.0,615561535.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,1.666196e+09,Warning: penny boards will make you a faggot
1,853718218,True,golden,92,NaN,The tweet contains hate speech,0.7227,NaN,True,NaN,0.0,615561723.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.295121e+08,Fuck dykes
2,853718219,True,golden,86,NaN,The tweet contains hate speech,0.5229,NaN,True,NaN,0.0,615562039.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,3.956238e+08,@sizzurp__ @ILIKECATS74 @yoPapi_chulo @brandon...
3,853718220,True,golden,98,NaN,The tweet contains hate speech,0.5184,NaN,True,NaN,0.0,615562068.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,4.975147e+08,"""@jayswaggkillah: ""@JacklynAnnn: @jayswaggkill..."
4,853718221,True,golden,88,NaN,The tweet uses offensive language but not hate...,0.5185,NaN,True,NaN,0.0,615562488.0,golden,NaN,The tweet contains hate speech,The tweet contains hate speech\nThe tweet uses...,NaN,1.0,5.889236e+08,@Zhugstubble You heard me bitch but any way I'...


In [16]:
data.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'does_this_tweet_contain_hate_speech',
       'does_this_tweet_contain_hate_speech:confidence', '_created_at',
       'orig__golden', 'orig__last_judgment_at', 'orig__trusted_judgments',
       'orig__unit_id', 'orig__unit_state', '_updated_at',
       'orig_does_this_tweet_contain_hate_speech',
       'does_this_tweet_contain_hate_speech_gold',
       'does_this_tweet_contain_hate_speech_gold_reason',
       'does_this_tweet_contain_hate_speechconfidence', 'tweet_id',
       'tweet_text'],
      dtype='object')

In [17]:
X = data['tweet_text']
X.shape

(14509,)

In [31]:
target = pd.factorize(data['does_this_tweet_contain_hate_speech'])

In [36]:
labels = target[0]
data['label'] = labels

In [41]:
print(data['does_this_tweet_contain_hate_speech'].value_counts())
print(data['label'].value_counts())

The tweet is not offensive                               7274
The tweet uses offensive language but not hate speech    4836
The tweet contains hate speech                           2399
Name: does_this_tweet_contain_hate_speech, dtype: int64
2    7274
0    4836
1    2399
Name: label, dtype: int64


In [42]:
y = data['label']
y.shape

(14509,)

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [45]:
# import and instantiate CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [46]:
# fit and transform X_train into X_train_dtm
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(10881, 23175)

In [47]:
# transform X_test into X_test_dtm
X_test_dtm = vect.transform(X_test)
X_test_dtm.shape

(3628, 23175)

In [48]:
# import and instantiate MultinomialNB
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [49]:
# train the model using X_train_dtm
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [50]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [51]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.74310915104740904

In [53]:
# calculate the null accuracy
y_test.value_counts().head(1) / y_test.shape

2    0.497519
Name: label, dtype: float64

In [54]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_class)

array([[ 958,   77,  182],
       [ 429,   98,   79],
       [ 161,    4, 1640]])

In [55]:
# print the classification report
print(metrics.classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

          0       0.62      0.79      0.69      1217
          1       0.55      0.16      0.25       606
          2       0.86      0.91      0.89      1805

avg / total       0.73      0.74      0.71      3628



In [60]:
# import and instantiate Random Forest
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(n_estimators=100, random_state=42)

In [61]:
rdf.fit(X_train_dtm, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)

In [62]:
y_pred_class = rdf.predict(X_test_dtm)

In [63]:
metrics.accuracy_score(y_test, y_pred_class)

0.75744211686879825

In [64]:
print(metrics.classification_report(y_test, y_pred_class))

             precision    recall  f1-score   support

          0       0.69      0.67      0.68      1217
          1       0.53      0.27      0.36       606
          2       0.83      0.98      0.90      1805

avg / total       0.73      0.76      0.73      3628

